# **Python、NumPy 與向量化 (Vectorization)**

本實驗簡要介紹本課程中會用到的一些科學運算工具，特別是 NumPy 科學運算套件，以及如何在 Python 中使用它。

# **1. 大綱**
- [&nbsp;&nbsp;1.1 目標](#toc_40015_1.1)
- [&nbsp;&nbsp;1.2 參考資料](#toc_40015_1.2)
- [2 Python 與 NumPy <a name='Python and NumPy'></a>](#toc_40015_2)
- [3 向量（Vectors）](#toc_40015_3)
- [&nbsp;&nbsp;3.1 抽象概念](#toc_40015_3.1)
- [&nbsp;&nbsp;3.2 NumPy 陣列](#toc_40015_3.2)
- [&nbsp;&nbsp;3.3 向量的建立](#toc_40015_3.3)
- [&nbsp;&nbsp;3.4 向量運算](#toc_40015_3.4)
- [4 矩陣（Matrices）](#toc_40015_4)
- [&nbsp;&nbsp;4.1 抽象概念](#toc_40015_4.1)
- [&nbsp;&nbsp;4.2 NumPy 陣列](#toc_40015_4.2)
- [&nbsp;&nbsp;4.3 矩陣的建立](#toc_40015_4.3)
- [&nbsp;&nbsp;4.4 矩陣運算](#toc_40015_4.4)


In [ ]:
import numpy as np
import time

#region 匯入相關模組

import sys, os
from pathlib import Path

def find_repo_root(marker="README.md"):
    cur = Path.cwd()
    while cur != cur.parent:  # 防止無限迴圈，到達檔案系統根目錄就停
        if (cur / marker).exists():
            return cur
        cur = cur.parent
    return None


def import_data_from_github():
    import urllib.request, shutil
    
    def isRunningInColab() -> bool:
        return "google.colab" in sys.modules

    def isRunningInJupyterLab() -> bool:
        try:
            import jupyterlab
            return True
        except ImportError:
            return False
        
    def detect_env():
        from IPython import get_ipython
        if isRunningInColab():
            return "Colab"
        elif isRunningInJupyterLab():
            return "JupyterLab"
        elif "notebook" in str(type(get_ipython())).lower():
            return "Jupyter Notebook"
        else:
            return "Unknown"
        
    def get_dir(env, dir_name): 
        if env == "Colab": 
            if "/content" not in sys.path:
                sys.path.insert(0, "/content")
            return f"/content/{dir_name}"
        else:
            return Path.cwd() / dir_name

    env = detect_env()
    DIR_NAME = "images"
    DIR_PATH = get_dir(env, DIR_NAME)
    REPO_DIR = "Machine-Learning-Lab"

    os.makedirs(DIR_PATH, exist_ok=True)

    BASE = f"https://raw.githubusercontent.com/mz038197/{REPO_DIR}/main"
    images_list = ["vectors.PNG", "dot_notrans.gif", "matrices.PNG"]
    for image in images_list:
        urllib.request.urlretrieve(f"{BASE}/lab/teacher/Regression/{DIR_NAME}/{image}", f"{DIR_PATH}/{image}")
   
repo_root = find_repo_root()

if repo_root is None:
    import_data_from_github()
    repo_root = Path.cwd()

os.chdir(repo_root)
print(f"✅ 切換工作目錄至 {Path.cwd()}")
sys.path.append(str(repo_root)) if str(repo_root) not in sys.path else None
print(f"✅ 加入到系統路徑")

#endregion 匯入相關模組


<a name="toc_40015_1.1"></a>
## 1.1 目標
在本實驗中，你將會：
- 複習在之前課程中使用到的 Python 與 NumPy 功能

<a name="toc_40015_1.2"></a>
## 1.2 實用參考資料
- NumPy 的官方文件（含基本介紹）：[NumPy.org](https://NumPy.org/doc/stable/)
- 進階主題：[*NumPy Broadcasting*（廣播機制）](https://NumPy.org/doc/stable/user/basics.broadcasting.html)


<br>

<a name="toc_40015_2"></a>
# **2. Python 與 NumPy** <a name='Python and NumPy'></a>
Python 是本課程使用的程式語言。它本身提供了多種數值型別與算術運算。

NumPy 則是在 Python 之上擴充的科學運算函式庫，提供更豐富的數值型別、向量、矩陣及許多矩陣相關函式。

NumPy 與 Python 可以非常順暢地搭配使用：Python 的算術運算子可以直接作用在 NumPy 的資料型別上，而許多 NumPy 函式也可以接受一般的 Python 數值型別作為輸入。


<br>

<a name="toc_40015_3"></a>
# **3 向量 (Vectors)**
<a name="toc_40015_3.1"></a>

## 3.1 概要

<img src="./images/vectors.PNG" style="width:386px;display: block; margin: 0 auto;" >

在本課程中，你所使用的「向量」，可以想成是**有順序的數字陣列**。在數學記號中，向量通常用粗體的小寫字母表示，例如 $\mathbf{x}$。

向量中的各個元素型別都必須相同；例如，一個向量不會同時包含字元和數值。

向量中元素的個數常被稱為向量的*維度（dimension）*，有些數學書也會用 *rank* 這個字。上圖中的向量維度是 $n$。

向量的元素可以透過索引（index）來存取：在數學裡索引通常從 1 到 n；在電腦科學與本系列實驗中，索引則是從 0 到 n-1。

以記號來說，若個別參照向量 $\mathbf{x}$ 的元素，會在下標標註索引，例如向量 $\mathbf{x}$ 的第 $0$ 個元素寫作 $x_0$。注意，這時的 $x$ 不再使用粗體。


<br>

<a name="toc_40015_3.2"></a>

## 3.2 NumPy 陣列

NumPy 的基本資料結構是可被索引（indexable）的 *n 維陣列（n-dimensional array）*，其中所有元素的型別（`dtype`）相同。你可能會注意到，我們再次使用了「維度（dimension）」這個字：
- 在上面談向量時，它代表「向量裡有多少個元素」。
- 在這裡，它代表「陣列有幾個索引維度」。

一維陣列（1-D array）只有一個索引。在課程 1 中，我們會用 NumPy 的一維陣列來表示向量。

- 一維陣列，形狀為 `(n,)`：包含 n 個元素，索引從 `[0]` 到 `[n-1]`
 

<br>

<a name="toc_40015_3.3"></a>
## 3.3 向量的建立


在 NumPy 中，用來建立資料的各種函式，通常會把「結果物件的形狀（shape）」當作第一個參數。對一維結果來說，這個參數可以是一個單一數值；對多維結果來說，則會是像 (n, m, ...) 這樣的 tuple，用來指定結果陣列的形狀。下面是幾個使用這些函式建立向量的例子。

In [ ]:
# NumPy routines which allocate memory and fill arrays with value
a = np.zeros(4);                print(f"np.zeros(4) :   a = {a}, a shape = {a.shape}, a data type = {a.dtype}")
a = np.zeros((4,));             print(f"np.zeros(4,) :  a = {a}, a shape = {a.shape}, a data type = {a.dtype}")
a = np.random.random_sample(4); print(f"np.random.random_sample(4): a = {a}, a shape = {a.shape}, a data type = {a.dtype}")

有些建立資料的函式並不接受 shape 這個 tuple 參數：

In [ ]:
# NumPy routines which allocate memory and fill arrays with value but do not accept shape as input argument
a = np.arange(4.);              print(f"np.arange(4.):     a = {a}, a shape = {a.shape}, a data type = {a.dtype}")
a = np.random.rand(4);          print(f"np.random.rand(4): a = {a}, a shape = {a.shape}, a data type = {a.dtype}")

當然，你也可以手動指定每一個元素的數值。 

In [ ]:
# NumPy routines which allocate memory and fill with user specified values
a = np.array([5,4,3,2]);  print(f"np.array([5,4,3,2]):  a = {a},     a shape = {a.shape}, a data type = {a.dtype}")
a = np.array([5.,4,3,2]); print(f"np.array([5.,4,3,2]): a = {a}, a shape = {a.shape}, a data type = {a.dtype}")

以上這些範例都建立了一個具有四個元素的一維向量 `a`。屬性 `a.shape` 會回傳其維度資訊。在這裡可以看到 `a.shape = (4,)`，表示這是一個含有 4 個元素的一維陣列。  

<a name="toc_40015_3.4"></a>
## 3.4 向量運算
接下來我們用幾個例子來看看向量上常見的運算。
<a name="toc_40015_3.4.1"></a>
### 3.4.1 索引（Indexing）
向量中的元素可以透過「索引（indexing）」與「切片（slicing）」來存取。NumPy 提供了非常完整的索引與切片功能；在這裡我們只會介紹本課程所需的基礎，更多細節可以參考官方文件：[Slicing and Indexing](https://NumPy.org/doc/stable/reference/arrays.indexing.html)。  
**Indexing（索引）**：依照陣列中元素的位置，取得**單一元素**。  
**Slicing（切片）**：依照索引範圍，取得**一個子集合（subset）**。  
在 NumPy 中，索引從 0 開始，因此向量 $\mathbf{a}$ 的第三個元素是 `a[2]`。

In [ ]:
#vector indexing operations on 1-D vectors
a = np.arange(10)
print(a)

#access an element
print(f"a[2].shape: {a[2].shape} a[2]  = {a[2]}, Accessing an element returns a scalar")

# access the last element, negative indexes count from the end
print(f"a[-1] = {a[-1]}")

#indexs must be within the range of the vector or they will produce and error
try:
    c = a[10]
except Exception as e:
    print("The error message you'll see is:")
    print(e)

<a name="toc_40015_3.4.2"></a>
### 3.4.2 切片（Slicing）
切片會使用三個值（`start:stop:step`）來建立一組索引。你也可以只提供其中一部分參數。以下透過範例來說明如何使用切片：

In [ ]:
#vector slicing operations
a = np.arange(10)
print(f"a         = {a}")

#access 5 consecutive elements (start:stop:step)
c = a[2:7:1];     print("a[2:7:1] = ", c)

# access 3 elements separated by two 
c = a[2:7:2];     print("a[2:7:2] = ", c)

# access all elements index 3 and above
c = a[3:];        print("a[3:]    = ", c)

# access all elements below index 3
c = a[:3];        print("a[:3]    = ", c)

# access all elements
c = a[:];         print("a[:]     = ", c)

<a name="toc_40015_3.4.3"></a>
### 3.4.3 單一向量運算
有許多實用的運算只牽涉到單一向量本身。

In [ ]:
a = np.array([1,2,3,4])
print(f"a             : {a}")
# negate elements of a
b = -a 
print(f"b = -a        : {b}")

# sum all elements of a, returns a scalar
b = np.sum(a) 
print(f"b = np.sum(a) : {b}")

b = np.mean(a)
print(f"b = np.mean(a): {b}")

b = a**2
print(f"b = a**2      : {b}")

<a name="toc_40015_3.4.4"></a>
### 3.4.4 向量與向量的逐元素運算
大多數 NumPy 的算術、邏輯與比較運算，同樣可以直接套用在向量上。這些運算子會**逐元素（element-wise）**地作用。例如：
$$ c_i = a_i + b_i $$

In [ ]:
a = np.array([ 1, 2, 3, 4])
b = np.array([-1,-2, 3, 4])
print(f"Binary operators work element wise: {a + b}")

當然，要讓這種運算正確進行，兩個向量必須擁有**相同的長度（size）**：

In [ ]:
#try a mismatched vector operation
c = np.array([1, 2])
try:
    d = a + c
except Exception as e:
    print("The error message you'll see is:")
    print(e)

<a name="toc_40015_3.4.5"></a>
### 3.4.5 純量與向量的運算
向量可以被「純量（scalar）」放大或縮小。純量就是一般的單一數值，這個數值會乘上向量中的所有元素。

In [ ]:
a = np.array([1, 2, 3, 4])

# multiply a by a scalar
b = 5 * a 
print(f"b = 5 * a : {b}")

<a name="toc_40015_3.4.6"></a>
### 3.4.6 向量與向量的內積（dot product）
內積（dot product）是線性代數與 NumPy 中非常核心的運算之一。在本課程中會大量使用到這個運算，因此務必要熟悉它。下圖展示了內積的形式：

<img src="./images/dot_notrans.gif" width=800 style="display: block; margin: 0 auto;"> 

內積會先將兩個向量對應位置的元素相乘，然後把所有乘積加總起來。
要能進行向量內積，兩個向量的維度（長度）必須相同。 

接下來我們自己實作一個內積函式：

**請使用 for 迴圈**，實作一個函式來回傳兩個向量的內積。給定輸入向量 $a$ 與 $b$，函式需要計算：
$$ x = \sum_{i=0}^{n-1} a_i b_i $$
假設 `a` 與 `b` 的形狀（shape）相同。

In [ ]:
def my_dot(a, b): 
    """
   Compute the dot product of two vectors
 
    Args:
      a (ndarray (n,)):  input vector 
      b (ndarray (n,)):  input vector with same dimension as a
    
    Returns:
      x (scalar): 
    """
    x=0
    for i in range(a.shape[0]):
        x = x + a[i] * b[i]
    return x

In [ ]:
# test 1-D
a = np.array([1, 2, 3, 4])
b = np.array([-1, 4, 3, 2])
print(f"my_dot(a, b) = {my_dot(a, b)}")

請注意，內積的結果應該是一個**純量（scalar）**。 

接下來我們用 `np.dot` 來做同樣的運算，比較結果。  

In [ ]:
# test 1-D
a = np.array([1, 2, 3, 4])
b = np.array([-1, 4, 3, 2])
c = np.dot(a, b)
print(f"NumPy 1-D np.dot(a, b) = {c}, np.dot(a, b).shape = {c.shape} ") 
c = np.dot(b, a)
print(f"NumPy 1-D np.dot(b, a) = {c}, np.dot(a, b).shape = {c.shape} ")


從上面的結果可以看到，在一維向量的情況下，`np.dot` 的結果與我們自己實作的內積是一致的。

<a name="toc_40015_3.4.7"></a>
### 3.4.7 速度的重要性：向量化 vs for 迴圈
我們之所以使用 NumPy，其中一個主要原因就是它在速度與記憶體效率上有明顯的優勢。下面用一個例子來示範：

In [ ]:
np.random.seed(1)
a = np.random.rand(10000000)  # very large arrays
b = np.random.rand(10000000)

tic = time.time()  # capture start time
c = np.dot(a, b)
toc = time.time()  # capture end time

print(f"np.dot(a, b) =  {c:.4f}")
print(f"Vectorized version duration: {1000*(toc-tic):.4f} ms ")

tic = time.time()  # capture start time
c = my_dot(a,b)
toc = time.time()  # capture end time

print(f"my_dot(a, b) =  {c:.4f}")
print(f"loop version duration: {1000*(toc-tic):.4f} ms ")

del(a);del(b)  #remove these big arrays from memory

可以看到，在這個例子中，向量化帶來了顯著的速度提升。這是因為 NumPy 能更有效地利用硬體中可用的資料平行性。現代的 CPU 與 GPU 都實作了「單指令多資料（SIMD, Single Instruction, Multiple Data）」的管線，允許同時對多筆資料執行相同的運算。當我們在機器學習中處理非常大型的資料集時，這種優化就變得格外關鍵。

<a name="toc_12345_3.4.8"></a>
### 3.4.8 本課程中的向量與向量運算
在課程 1 中，向量與向量之間的運算會非常常見，原因如下：
- 之後，我們會把訓練樣本存成一個維度為 (m, n) 的陣列 `X_train`。稍後會搭配範例詳細說明，這裡先注意：它是一個**二維陣列（矩陣）**（請參考下一節關於矩陣的說明）。
- `w` 會是一個形狀為 `(n,)` 的一維向量。
- 進行運算時，我們通常會在迴圈中逐一取出樣本，也就是對 `X` 做索引，例如：`X[i]`。
- `X[i]` 的回傳值形狀是 `(n,)`，也就是一維向量；因此，牽涉到 `X[i]` 的運算，往往就是向量與向量的運算。  

說明雖然有點長，但在進行向量運算時，對每個運算元（operand）的 shape 是否對齊、是否正確，保持清楚的理解是非常重要的。

In [ ]:
# show common Course 1 example
X = np.array([[1],[2],[3],[4]])
w = np.array([2])
c = np.dot(X[1], w)

print(f"X[1] has shape {X[1].shape}")
print(f"w has shape {w.shape}")
print(f"c has shape {c.shape}")
print(c)

<a name="toc_40015_4"></a>

# **4 矩陣(Matrices)**


<a name="toc_40015_4.1"></a>
## 4.1 概要
矩陣（matrix）可以看成是**二維陣列**。矩陣中的所有元素型別都必須相同。在數學記號中，矩陣通常以粗體的大寫字母表示，例如 $\mathbf{X}$。在本實驗與其他相關實驗中，`m` 通常代表列（row）的數目，`n` 則代表行（column）的數目。矩陣中的元素可以用二維索引來存取。在數學裡，索引通常從 1 到 n；而在電腦科學與本系列實驗中，索引則是從 0 到 n-1。 
 
<figure>
    <center> <img src="./images/matrices.PNG"  alt='missing'  width=900><center/>
    <figcaption> 一般矩陣表示法：第一個索引是列（row），第二個索引是行（column） </figcaption>
<figure/>

<a name="toc_40015_4.2"></a>
## 4.2 NumPy 陣列

NumPy 的基本資料結構是可被索引的 *n 維陣列*，其中所有元素的型別（`dtype`）相同，這在前面已經介紹過。矩陣則是具有二維索引 [m, n] 的陣列。

在課程 1 中，二維矩陣主要用來儲存訓練資料。假設有 $m$ 個樣本、每個樣本有 $n$ 個特徵，則可以用一個形狀為 (m, n) 的陣列來表示這組訓練資料。課程 1 並不會大量直接在整個矩陣上做運算，而是經常先從矩陣中取出單一樣本（形成一個向量），再對該向量進行運算。接下來你會複習：
- 如何建立這些資料
- 如何對矩陣做切片與索引

<a name="toc_40015_4.3"></a>
## 4.3 矩陣的建立
前面用來建立一維向量的那些函式，同樣也可以用來建立二維或更高維度的陣列。以下是幾個範例：


在下面的例子中，我們透過傳入 shape 這個 tuple 來建立二維結果。請注意 NumPy 使用多層中括號來表示不同維度；另外，在輸出時，NumPy 會將每一列（row）印在一行上。

In [ ]:
a = np.zeros((1, 5))                                       
print(f"a shape = {a.shape}, a = {a}")                     

a = np.zeros((2, 1))                                                                   
print(f"a shape = {a.shape}, a = {a}") 

a = np.random.random_sample((1, 1))  
print(f"a shape = {a.shape}, a = {a}") 

你也可以手動指定矩陣中的資料。各維度則透過額外的一層層中括號來表示，其格式與上面 NumPy 印出的結果相同。

In [ ]:
# NumPy routines which allocate memory and fill with user specified values
a = np.array([[5], [4], [3]]);   print(f" a shape = {a.shape}, np.array: a = {a}")
a = np.array([[5],   # One can also
              [4],   # separate values
              [3]]); #into separate rows
print(f" a shape = {a.shape}, np.array: a = {a}")

<a name="toc_40015_4.4"></a>
## 4.4 矩陣運算
接下來我們用幾個例子來看看矩陣上的常見運算。

<a name="toc_40015_4.4.1"></a>
### 4.4.1 索引（Indexing）


矩陣多了一個索引維度，兩個索引分別表示 [列, 行]（[row, column]）。依照使用方式不同，存取結果可以是矩陣中的單一元素，或是一整列／一整行。如下所示：

In [ ]:
#vector indexing operations on matrices
a = np.arange(6).reshape(-1, 2)   #reshape is a convenient way to create matrices
print(f"a.shape: {a.shape}, \na= {a}")

#access an element
print(f"\na[2,0].shape:   {a[2, 0].shape}, a[2,0] = {a[2, 0]},     type(a[2,0]) = {type(a[2, 0])} Accessing an element returns a scalar\n")

#access a row
print(f"a[2].shape:   {a[2].shape}, a[2]   = {a[2]}, type(a[2])   = {type(a[2])}")

特別值得注意的是最後一個例子：只指定列索引來存取矩陣時，回傳的會是**一維向量（1-D vector）**。

**Reshape（重塑形狀）**  
前一個例子使用了 [reshape](https://numpy.org/doc/stable/reference/generated/numpy.reshape.html) 來調整陣列的形狀：  
`a = np.arange(6).reshape(-1, 2) `   
這行程式碼首先建立了一個包含 6 個元素的**一維向量**，接著利用 `reshape` 指令把它變成一個**二維陣列**。同樣的結果也可以寫成：  
`a = np.arange(6).reshape(3, 2) `  
得到一個 3 列、2 行的陣列。  
其中 `-1` 這個引數的意思是：請函式根據陣列總長度與給定的欄數，自動推算出適合的列數。

<a name="toc_40015_4.4.2"></a>
### 4.4.2 切片（Slicing）
切片會使用三個值（`start:stop:step`）來建立索引區間，你也可以只提供其中一部分參數。以下透過範例來說明切片在二維矩陣上的用法：

In [ ]:
#vector 2-D slicing operations
a = np.arange(20).reshape(-1, 10)
print(f"a = \n{a}")

#access 5 consecutive elements (start:stop:step)
print("a[0, 2:7:1] = ", a[0, 2:7:1], ",  a[0, 2:7:1].shape =", a[0, 2:7:1].shape, "a 1-D array")

#access 5 consecutive elements (start:stop:step) in two rows
print("a[:, 2:7:1] = \n", a[:, 2:7:1], ",  a[:, 2:7:1].shape =", a[:, 2:7:1].shape, "a 2-D array")

# access all elements
print("a[:,:] = \n", a[:,:], ",  a[:,:].shape =", a[:,:].shape)

# access all elements in one row (very common usage)
print("a[1,:] = ", a[1,:], ",  a[1,:].shape =", a[1,:].shape, "a 1-D array")
# same as
print("a[1]   = ", a[1],   ",  a[1].shape   =", a[1].shape, "a 1-D array")


<a name="toc_40015_5.0"></a>
## 恭喜！
在本實驗中，你已經掌握了課程 1 所需要用到的 Python 與 NumPy 相關功能。